In [1]:
import pandas as pd

In [34]:
import json
def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print("File not found.")
        return None
    except json.JSONDecodeError:
        print("Invalid JSON format.")
        return None

results = []
question_id = 0
for category in ["moral"]: #"causal"]:
    data = read_json_file(f'/fsx-checkpoints/bhakti/raw_datasets/moca/data/{category}_dataset_v1.json')

    for index in range(0, len(data),1):
        row1 = json.loads(data[index])
        if category == "causal" and not row1["is_ambiguous"]:
                 # filtering only ambiguous ones for causal due to big size 
                 continue
        user_prompt = row1["story"] + "\n" + row1["question"]
        answer1 = "Yes"
        answer2 = "No"
        human_vote1 = row1["answer_dist"][0]
        human_vote2 = row1["answer_dist"][1]
        print(float(human_vote1),float(human_vote2))
        if float(human_vote1)> float(human_vote2):
            results.append({"id": question_id, "question": user_prompt, "correct_answer": answer1, "incorrect_answer": answer2,  "correct_human_vote":human_vote1, "incorrect_human_vote": human_vote2,})
        else:
            results.append({"id": question_id, "question": user_prompt, "correct_answer": answer2, "incorrect_answer": answer1,  "correct_human_vote":human_vote2, "incorrect_human_vote": human_vote1,})
        question_id+=1

0.64 0.36
0.64 0.36
0.44 0.56
0.52 0.48
0.56 0.43999999999999995
0.52 0.48
0.48 0.52
0.68 0.31999999999999995
0.64 0.36
0.56 0.43999999999999995
0.48 0.52
0.2 0.8
0.64 0.36
0.6 0.4
0.72 0.28
0.6 0.4
0.28 0.72
0.72 0.28
0.48 0.52
0.4 0.6
0.4 0.6
0.8 0.19999999999999996
0.2 0.8
0.44 0.56
0.56 0.43999999999999995
0.52 0.48
0.68 0.31999999999999995
0.6 0.4
0.6 0.4
0.76 0.24
0.76 0.24
0.8 0.19999999999999996
0.6 0.4
0.68 0.31999999999999995
0.72 0.28
0.72 0.28
0.52 0.48
0.64 0.36
0.08 0.92
0.16 0.84
0.6 0.4
0.32 0.6799999999999999
0.56 0.43999999999999995
0.8 0.19999999999999996
0.4 0.6
0.56 0.43999999999999995
0.4 0.6
0.56 0.43999999999999995
0.92 0.07999999999999996
0.64 0.36
0.56 0.43999999999999995
0.64 0.36
0.16 0.84
0.6 0.4
0.4 0.6
0.68 0.31999999999999995
0.32 0.6799999999999999
0.32 0.6799999999999999
0.56 0.43999999999999995
0.16 0.84
0.8 0.19999999999999996
0.72 0.28


In [36]:
df = pd.DataFrame(results)
df = df.drop_duplicates('id')
df = df.rename(columns={"correct_answer": "correct answer"})
df = df.rename(columns={"incorrect_answer": "negative answer"})
df["tokens"] = 50
df

,id,question,correct answer,negative answer,correct_human_vote,incorrect_human_vote,tokens
0,0,A high-speed train is about to hit a bus with ...,Yes,No,0.64,0.36,50
1,1,A high-speed train is about to hit a bus with ...,Yes,No,0.64,0.36,50
2,2,"In a restaurant, a bomb threatens to kill 9 gu...",No,Yes,0.56,0.44,50
3,3,"In a restaurant, a bomb threatens to kill 9 gu...",Yes,No,0.52,0.48,50
4,4,A virus causing paraplegia threatens 4 patient...,Yes,No,0.56,0.44,50
...,...,...,...,...,...,...,...
57,57,You are a miner. The only way out of the mine ...,No,Yes,0.68,0.32,50
58,58,You are an electrician hired to fix an electri...,Yes,No,0.56,0.44,50
59,59,You are an electrician hired to fix an electri...,No,Yes,0.84,0.16,50
60,60,You are a military officer during a war. Your ...,Yes,No,0.80,0.20,50


In [39]:
df.to_csv("/home/bhakti/llm_debate/exp/moral/debate_sim/data0.csv")

In [ ]:
id,question,correct answer,negative answer,complete,transcript,answer,prompt,cot prompt,story,story_title,question_set_id,story tokens


In [18]:
unique_questions_df["correct_answer"].value_counts

<bound method IndexOpsMixin.value_counts of 0     Yes
1     Yes
2     Yes
3     Yes
4     Yes
     ... 
57    Yes
58    Yes
59    Yes
60    Yes
61    Yes
Name: correct_answer, Length: 62, dtype: object>

In [19]:
for question in df['question'].unique():
    yes_votes = df[(df['question'] == question) & (df['answer'] == 'Yes')]['human_vote']
    no_votes = df[(df['question'] == question) & (df['answer'] == 'No')]['human_vote']
    
    assert (yes_votes.min() >= no_votes.max()), f"Human vote for 'yes' is not greater than human vote for 'no' for question '{question}'"

AssertionError: Human vote for 'yes' is not greater than human vote for 'no' for question 'In a restaurant, a bomb threatens to kill 9 guests. The bomb could be thrown onto the patio, where 1 guest would be killed.
Should the bomb be thrown to the patio?'

In [2]:
df = pd.read_csv("/home/bhakti/llm_debate/exp/minimal/debate_sim/data0.csv")

In [3]:
df

,id,question,correct answer,negative answer,complete,transcript,answer,prompt,cot prompt,story,story_title,question_set_id,story tokens
0,0,Why did the Earth doctor use the mortar and pe...,To help the local doctor understand the treatment,To keep the IV drip going,True,"{""index"": 0, ""question"": ""Why did the Earth do...",NaN,NaN,NaN,R\n X\nBY ALAN E. NOURSE\nThe tenth son of a t...,Rx,60412_XM0T4STT,6471
1,1,Which best describes the relationship between ...,They have similar goals but do not necessarily...,"Neither of them likes the other, in a way that...",True,"{""index"": 1, ""question"": ""Which best describes...",NaN,NaN,NaN,Transcriber's Note:\nThis etext was produced f...,To Each His Star,31355_N2HLEA08,6302
2,2,How does Bergstrom feel about Zarwell?,Bergstrom admires Zarwell. He wants Zarwell to...,Bergstrom thinks Zarwell is a dangerous man. H...,True,"{""index"": 2, ""question"": ""How does Bergstrom f...",NaN,NaN,NaN,Transcriber’s note:\nThis story was published ...,Monkey On His Back,26569_CEKEK4QL,6801
3,3,Who is the protagonist of the story and what i...,"An ex soldier who fought in World War III, tra...","An ex soldier who fought in World War II, trav...",True,"{""index"": 3, ""question"": ""Who is the protagoni...",NaN,NaN,NaN,HOMECOMING\nBY MIGUEL HIDALGO\nWhat lasts fore...,Homecoming,60515_4ZLFFIZT,3919
4,4,What is Paul doing in Tangier?,He is on a business trip to find a source of p...,He is on a business trip scouting locations fo...,True,"{""index"": 4, ""question"": ""What is Paul doing i...",NaN,NaN,NaN,One can't be too cautious about the\n \n peop...,I'm a Stranger Here Myself,26741_OVYFBIST,2827
5,5,What is the significance of the narrator’s hei...,It shows he is not from Dondromogon\n,It shows he is the Conquering Stranger \n,True,"{""index"": 5, ""question"": ""What is the signific...",NaN,NaN,NaN,Warrior of Two Worlds\nBy MANLY WADE WELLMAN\n...,Warrior of Two Worlds,63097_4CW2KAPS,6798
6,6,How did Ninon’s travel companion fare?,He was reduced to particles,He became more youthful until a baby and then ...,True,"{""index"": 6, ""question"": ""How did Ninon\u2019s...",NaN,NaN,NaN,"TIME and the WOMAN\nBy Dewey, G. Gordon\n[Tran...",Time and the Woman,40965_7AWX7OE9,6547
7,7,What is Pashkov’s assignment?,To kidnap Colonel James and then impersonate h...,To kill Colonel James.,True,"{""index"": 7, ""question"": ""What is Pashkov\u201...",NaN,NaN,NaN,THE COOL WAR\nby ANDREW FETLER\n\n\n Illustrat...,The Cool War,51256_P2M1I2KR,8199
8,8,Why does no one on town know who Haron Gorka is?,"He’s not a real resident, but an interstellar ...",He’s not a real resident. He’s using a fake na...,True,"{""index"": 8, ""question"": ""Why does no one on t...",NaN,NaN,NaN,PEN PAL\nIllustrated by DON SIBLEY\n\n\n By MI...,Pen Pal,51286_56I6DUG4,7164
9,9,Which is the best summary of this story?,A man realizes his obliviousness and shifts hi...,A man loses hope for his world and gives up in...,True,"{""index"": 9, ""question"": ""Which is the best su...",NaN,NaN,NaN,MONOPOLY\nBy Vic Phillips and Scott Roberts\nS...,Monopoly,61499_93EZLOIH,6850
